In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [12]:
import graphlib

queue = graphlib.TopologicalSorter()

graph = {"D": {"B", "C"}, "C": {"A"}, "B": {"A"}}
sorter = graphlib.TopologicalSorter(graph)
sorter.prepare()
while sorter.is_active():
    for node in sorter.get_ready():
        print(node)
        sorter.done(node)

A
C
B
D


In [16]:
import operator

# item access
data = [1, 2, 3]
get = operator.itemgetter(slice(1, 2))
data = get(data)

[2]